In [1]:
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load data
data = []
with open('../data/HCD35_pos.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [3]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [4]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [5]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [6]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

In [7]:
# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD35_pos.h5')

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.9805 - acc: 0.2694 - val_loss: 1.4347 - val_acc: 0.2827
Epoch 2/100
1340/1340 [==============================] - 1s 638us/step - loss: 1.8146 - acc: 0.2687 - val_loss: 1.4782 - val_acc: 0.3244
Epoch 3/100
1340/1340 [==============================] - 1s 635us/step - loss: 1.7504 - acc: 0.3030 - val_loss: 1.4241 - val_acc: 0.3333
Epoch 4/100
1340/1340 [==============================] - 1s 644us/step - loss: 1.6849 - acc: 0.3104 - val_loss: 1.3657 - val_acc: 0.3512
Epoch 5/100
1340/1340 [==============================] - 1s 649us/step - loss: 1.7023 - acc: 0.3172 - val_loss: 1.2952 - val_acc: 0.3899
Epoch 6/100
1340/1340 [==============================] - 1s 655us/step - loss: 1.5395 - acc: 0.3343 - val_loss: 1.2399 - val_acc: 0.4137
Epoch 7/100
1340/1340 [==============================] - 1s 670us/step - loss: 1.5224 - acc: 0.3478 - val_loss: 1.2341 - val_acc: 0.4

In [8]:
#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

j = []
elapsed_time = time.time() - start
j.append(elapsed_time)

f = []
f.append(score[1]*100)

In [9]:
score[1]

0.6755952380952381

In [10]:
# load data
data = []
with open('../data/HCD45_pos.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [11]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [12]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [13]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [14]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD45_pos.h5')

#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

elapsed_time = time.time() - start
j.append(elapsed_time)

f.append(score[1]*100)

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 4s 3ms/step - loss: 1.9418 - acc: 0.2560 - val_loss: 1.4291 - val_acc: 0.2917
Epoch 2/100
1340/1340 [==============================] - 1s 1ms/step - loss: 1.8003 - acc: 0.2642 - val_loss: 1.4512 - val_acc: 0.3393
Epoch 3/100
1340/1340 [==============================] - 1s 990us/step - loss: 1.7191 - acc: 0.2978 - val_loss: 1.4419 - val_acc: 0.3571
Epoch 4/100
1340/1340 [==============================] - 1s 995us/step - loss: 1.6577 - acc: 0.2754 - val_loss: 1.3741 - val_acc: 0.3601
Epoch 5/100
1340/1340 [==============================] - 1s 1ms/step - loss: 1.6417 - acc: 0.3067 - val_loss: 1.3110 - val_acc: 0.3631
Epoch 6/100
1340/1340 [==============================] - 1s 988us/step - loss: 1.5214 - acc: 0.3284 - val_loss: 1.2601 - val_acc: 0.5000
Epoch 7/100
1340/1340 [==============================] - 1s 967us/step - loss: 1.5267 - acc: 0.3627 - val_loss: 1.2241 - val_acc: 0.4256


1340/1340 [==============================] - 2s 1ms/step - loss: 0.2662 - acc: 0.9007 - val_loss: 0.8928 - val_acc: 0.6577
Epoch 61/100
1340/1340 [==============================] - 2s 1ms/step - loss: 0.2616 - acc: 0.9194 - val_loss: 0.8822 - val_acc: 0.6667
Epoch 62/100
1340/1340 [==============================] - 2s 1ms/step - loss: 0.2481 - acc: 0.9037 - val_loss: 0.8698 - val_acc: 0.6696
Epoch 63/100
1340/1340 [==============================] - 2s 1ms/step - loss: 0.2560 - acc: 0.9090 - val_loss: 0.8715 - val_acc: 0.6667
Epoch 64/100
1340/1340 [==============================] - 2s 1ms/step - loss: 0.2442 - acc: 0.9179 - val_loss: 0.8715 - val_acc: 0.6756
Epoch 65/100
1340/1340 [==============================] - 2s 1ms/step - loss: 0.2380 - acc: 0.9194 - val_loss: 0.8648 - val_acc: 0.6815
Epoch 66/100
1340/1340 [==============================] - 2s 1ms/step - loss: 0.2210 - acc: 0.9231 - val_loss: 0.8528 - val_acc: 0.7083
Epoch 67/100
1340/1340 [==============================] - 2s 

In [15]:
# load data
data = []
with open('../data/HCD65_pos.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [16]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [17]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [18]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [19]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD65_pos.h5')

#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

elapsed_time = time.time() - start
j.append(elapsed_time)

f.append(score[1]*100)

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.9229 - acc: 0.2701 - val_loss: 1.4263 - val_acc: 0.2798
Epoch 2/100
1340/1340 [==============================] - 1s 794us/step - loss: 1.7823 - acc: 0.2634 - val_loss: 1.4422 - val_acc: 0.3185
Epoch 3/100
1340/1340 [==============================] - 1s 761us/step - loss: 1.7569 - acc: 0.2903 - val_loss: 1.4050 - val_acc: 0.3304
Epoch 4/100
1340/1340 [==============================] - 1s 758us/step - loss: 1.6661 - acc: 0.2769 - val_loss: 1.3725 - val_acc: 0.3423
Epoch 5/100
1340/1340 [==============================] - 1s 818us/step - loss: 1.6190 - acc: 0.3104 - val_loss: 1.3171 - val_acc: 0.3571
Epoch 6/100
1340/1340 [==============================] - 1s 829us/step - loss: 1.5514 - acc: 0.3142 - val_loss: 1.2756 - val_acc: 0.3690
Epoch 7/100
1340/1340 [==============================] - 1s 824us/step - loss: 1.5300 - acc: 0.3396 - val_loss: 1.2414 - val_acc: 0.4

1340/1340 [==============================] - 2s 1ms/step - loss: 0.5285 - acc: 0.7851 - val_loss: 0.9313 - val_acc: 0.6696
Epoch 61/100
1340/1340 [==============================] - 2s 1ms/step - loss: 0.5117 - acc: 0.8075 - val_loss: 0.9198 - val_acc: 0.6577
Epoch 62/100
1340/1340 [==============================] - 2s 1ms/step - loss: 0.4774 - acc: 0.8090 - val_loss: 0.9059 - val_acc: 0.6696
Epoch 63/100
1340/1340 [==============================] - 3s 2ms/step - loss: 0.4638 - acc: 0.8284 - val_loss: 0.8948 - val_acc: 0.6786
Epoch 64/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.4791 - acc: 0.8201 - val_loss: 0.8855 - val_acc: 0.6964
Epoch 65/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.4670 - acc: 0.8090 - val_loss: 0.8807 - val_acc: 0.7083
Epoch 66/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.4757 - acc: 0.8097 - val_loss: 0.8830 - val_acc: 0.6756
Epoch 67/100
1340/1340 [==============================] - 2s 

In [20]:
# load data
data = []
with open('../data/HCD35_neg.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [21]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [22]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [23]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [24]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD35_neg.h5')

#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

elapsed_time = time.time() - start
j.append(elapsed_time)

f.append(score[1]*100)

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 6ms/step - loss: 2.0533 - acc: 0.3351 - val_loss: 1.2986 - val_acc: 0.4362
Epoch 2/100
373/373 [==============================] - 0s 235us/step - loss: 1.8480 - acc: 0.3029 - val_loss: 1.2835 - val_acc: 0.4362
Epoch 3/100
373/373 [==============================] - 0s 225us/step - loss: 1.8657 - acc: 0.2895 - val_loss: 1.3194 - val_acc: 0.4574
Epoch 4/100
373/373 [==============================] - 0s 229us/step - loss: 1.9364 - acc: 0.3003 - val_loss: 1.3557 - val_acc: 0.4574
Epoch 5/100
373/373 [==============================] - 0s 238us/step - loss: 1.6219 - acc: 0.3512 - val_loss: 1.3658 - val_acc: 0.4468
Epoch 6/100
373/373 [==============================] - 0s 243us/step - loss: 1.6639 - acc: 0.3834 - val_loss: 1.3645 - val_acc: 0.4574
Epoch 7/100
373/373 [==============================] - 0s 233us/step - loss: 1.7337 - acc: 0.3780 - val_loss: 1.3318 - val_acc: 0.4894
Epoch 8/100


In [25]:
# load data
data = []
with open('../data/HCD45_neg.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [26]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [27]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [28]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [29]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD45_neg.h5')

#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

elapsed_time = time.time() - start
j.append(elapsed_time)

f.append(score[1]*100)

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 3s 7ms/step - loss: 2.0206 - acc: 0.2869 - val_loss: 1.3158 - val_acc: 0.4362
Epoch 2/100
373/373 [==============================] - 0s 273us/step - loss: 1.8925 - acc: 0.3056 - val_loss: 1.2813 - val_acc: 0.4468
Epoch 3/100
373/373 [==============================] - 0s 265us/step - loss: 1.8854 - acc: 0.2949 - val_loss: 1.3499 - val_acc: 0.4362
Epoch 4/100
373/373 [==============================] - 0s 264us/step - loss: 1.7426 - acc: 0.3566 - val_loss: 1.3973 - val_acc: 0.4574
Epoch 5/100
373/373 [==============================] - 0s 283us/step - loss: 1.6474 - acc: 0.3727 - val_loss: 1.4106 - val_acc: 0.4574
Epoch 6/100
373/373 [==============================] - 0s 272us/step - loss: 1.5675 - acc: 0.3834 - val_loss: 1.3840 - val_acc: 0.4574
Epoch 7/100
373/373 [==============================] - 0s 268us/step - loss: 1.6529 - acc: 0.3646 - val_loss: 1.3524 - val_acc: 0.4574
Epoch 8/100


In [30]:
# load data
data = []
with open('../data/HCD65_neg.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [31]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [32]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [33]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [34]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD65_neg.h5')

#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

elapsed_time = time.time() - start
j.append(elapsed_time)

f.append(score[1]*100)

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 3s 8ms/step - loss: 2.0837 - acc: 0.2869 - val_loss: 1.3364 - val_acc: 0.4043
Epoch 2/100
373/373 [==============================] - 0s 427us/step - loss: 1.9090 - acc: 0.2976 - val_loss: 1.2452 - val_acc: 0.4255
Epoch 3/100
373/373 [==============================] - 0s 425us/step - loss: 1.9388 - acc: 0.2976 - val_loss: 1.2644 - val_acc: 0.5106
Epoch 4/100
373/373 [==============================] - 0s 421us/step - loss: 1.8523 - acc: 0.3190 - val_loss: 1.3075 - val_acc: 0.5106
Epoch 5/100
373/373 [==============================] - 0s 430us/step - loss: 1.7209 - acc: 0.3485 - val_loss: 1.3087 - val_acc: 0.5106
Epoch 6/100
373/373 [==============================] - 0s 419us/step - loss: 1.6821 - acc: 0.3512 - val_loss: 1.2787 - val_acc: 0.5106
Epoch 7/100
373/373 [==============================] - 0s 415us/step - loss: 1.6443 - acc: 0.3592 - val_loss: 1.2445 - val_acc: 0.5106
Epoch 8/100


Epoch 61/100
373/373 [==============================] - 0s 420us/step - loss: 0.7082 - acc: 0.6917 - val_loss: 0.9319 - val_acc: 0.6383
Epoch 62/100
373/373 [==============================] - 0s 425us/step - loss: 0.7825 - acc: 0.6756 - val_loss: 0.9322 - val_acc: 0.6383
Epoch 63/100
373/373 [==============================] - 0s 413us/step - loss: 0.6933 - acc: 0.6756 - val_loss: 0.9341 - val_acc: 0.6383
Epoch 64/100
373/373 [==============================] - 0s 420us/step - loss: 0.7223 - acc: 0.7024 - val_loss: 0.9366 - val_acc: 0.6489
Epoch 65/100
373/373 [==============================] - 0s 418us/step - loss: 0.6697 - acc: 0.7239 - val_loss: 0.9395 - val_acc: 0.6489
Epoch 66/100
373/373 [==============================] - 0s 418us/step - loss: 0.6740 - acc: 0.7131 - val_loss: 0.9437 - val_acc: 0.6489
Epoch 67/100
373/373 [==============================] - 0s 416us/step - loss: 0.6614 - acc: 0.7265 - val_loss: 0.9486 - val_acc: 0.6596
Epoch 68/100
373/373 [==========================

In [35]:
pd.DataFrame(
    [f, j], 
    index=['Accuracy', 'Time'],
    columns = [
        'HCD35_Positive','HCD45_Positive','HCD65_Positive',
        'HCD35_Negative','HCD45_Negative','HCD65_Negative'
    ]
).T.to_csv('../result/Keras.csv')